### All augumentation (except horizontal_flip) turned off. Trying to overfit.

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
# Global var definitions

img_width, img_height = 150, 150
train_data_dir = "X:/train-scene classification/data3/train/"
validation_data_dir = "X:/train-scene classification/data3/valid/"
test_data_dir = "X:/train-scene classification/data3/test/"
nb_train_samples = 16384
nb_validation_samples = 1600 
batch_size = 16
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/train-scene classification/chkpts/NASNetMobile_2.h5" # Change for every new training session 
NASNetLarge_save_dir = 'X:/train-scene classification/NASNetLarge.h5'
NASNetMobile_save_dir = 'X:/train-scene classification/NASNetMobile.h5'
NASNetMobile_overfit_save_dir = "NASNetMobile-overfit-99-percent-training-cat_acc.h5"
architecture_plot_save_dir = 'X:/train-scene classification/nasnetmobile-base_model.png'

In [ ]:
# base_model = NASNetMobile(input_shape=(img_width, img_height, 3), include_top=False, weights='imagenet', input_tensor=None, pooling='avg', classes=1000)

In [ ]:
# base_model.save(NASNetMobile_save_dir)
base_model = load_model(NASNetMobile_save_dir)

In [ ]:
base_model.summary()

In [ ]:
# plot_model(base_model, to_file=architecture_plot_save_dir)

In [ ]:
for layer in base_model.layers[:-90]:
    layer.trainable = False

base_model.summary()
print(len(base_model.layers))

In [ ]:
x = base_model.output
x = Dense(1024, activation="relu")(x)
x = Dropout(0.0)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.0)(x)
predictions = Dense(6, activation="softmax")(x)

In [ ]:
# creating the final model 
model = Model(input = base_model.input, output = predictions)
model.summary()

In [ ]:
# compile the model 
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=["categorical_accuracy"])
# decay = 1e-6 <--- use this for Adam decay

In [ ]:
# model_final.load_weights(chkpt_dir)

In [3]:
model = load_model(NASNetMobile_overfit_save_dir)

In [4]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
shear_range = 0, # 10
fill_mode = "reflect",
zoom_range = 0.0, # 0.4
width_shift_range = 0.0, # 0.3
height_shift_range=0.0, # 0.3
# brightness_range = (0.75, 1.25),
rotation_range=0) # 10)

validation_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
shear_range = 0, # 10
fill_mode = "reflect",
zoom_range = 0.0, # 0.4
width_shift_range = 0.0, # 0.3
height_shift_range=0.0, # 0.3
# brightness_range = (0.75, 1.25),
rotation_range=0) # 10)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 15434 images belonging to 6 classes.
Found 1600 images belonging to 6 classes.


In [5]:
# Save the model according to the conditions  
# chkpt_dir = "X:/train-scene classification/chkpts/densenet_extra_fc.h5" # Change for every new training session 

checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, patience=500, verbose=1, mode='auto')
#[TrainValTensorBoard(write_graph=False)]

In [6]:
# Train the model 
model.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1000, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=1024, validation_steps=1600)`
  


Epoch 1/1000
 115/1024 [==>...........................] - ETA: 1:27 - loss: 0.0376 - categorical_accuracy: 0.9891

KeyboardInterrupt: 

In [ ]:
# Saving Model
model.save(NASNetMobile_overfit_save_dir)